<a href="https://colab.research.google.com/github/Shabbarsyed/Cash-Ratio-Analysis/blob/main/Similarity_check_URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
from google.cloud import bigquery

In [ ]:
from pandas.io import gbq

In [ ]:
df=gbq.read_gbq("select * FROM similaritycheck.urls ",project_id="sustained-works-329217")

In [ ]:
#importing all libraries
import pandas as pd
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Removing Unnecessary Data Column
# Removing Unnecessary Data Column
df= df.iloc[:, [1,6]]

In [ ]:
#the columns that we needed are
df=df.dropna()

In [ ]:
# Chaning Column name TO Url2 To follow the pattern of ALgorithum we have created
df.rename(columns={"ems_page_url": "Url2"}, inplace=True)
df.rename(columns={"text": "Url2text"}, inplace=True)

In [ ]:
# Now removing the unecessary text from url so we can configure that we are not
# comparing the same url
for i in range(len(df)):
    df.loc[i,'Url'] = df.loc[i,'Url2'].replace("https://", "")


In [ ]:
for i in range(len(df)):
    df['Content'] = df['Url'].apply(lambda url: url.split('/', 1)[0])

In [ ]:
# User input here
Url1="https://www.bettycrocker.co.uk"
Url2=Url1.replace("https://", "")

Mainurl=df[df['Content'] == Url2 ]
# removing all the urls that are related to the Urls user enter
df = df[df['Content'] != Url2 ]

In [ ]:
# reseting the indexes
df = df.reset_index(drop=True)

# reseting the indexes
Mainurl= Mainurl.reset_index(drop=True)

In [ ]:
import re

def remove_special_characters_keep_spaces(input_string):
    return re.sub('[^A-Za-z0-9\s]+', '', input_string)
# here we have removed all the special character from our main website link that we are going to compare with
# other websites
for i in range(len(Mainurl)):
    input_string= Mainurl.loc[i,'Url2text']
    Mainurl.loc[i,'Url2text'] = remove_special_characters_keep_spaces(input_string)

In [ ]:
for j in range(len(df)):
      input_stringdf= df.loc[j,'Url2text']
      df.loc[j,'Url2text'] = remove_special_characters_keep_spaces(input_stringdf)

In [ ]:
#checking that all the special character has been removed or not
Mainurl.loc[100,'Url2text']

'chocolate chip marble cake take this chocolate chip marble cake round to share with friends  chocolate  birthday  cake  fathers day difficulty level     chocolate chip marble cake take this chocolate chip marble cake round to share with friends  2h 15min total time  12 servings servings all you need is   1 box betty crocker velvety vanilla cake mix  225g dark chocolate chips  90ml vegetable oil  180ml water  3 medium free range eggs icing  1 tub 400g betty crocker velvety vanilla icing made with these betty crocker  products  velvety vanilla cake mix  velvety vanilla icing bettys easy steps preheat the oven to 180c 160c for fan assisted ovensgas mark 4 grease and line a 20cm round cake tin combine 100g of the chocolate chips with 1 tablespoon of the dry cake mix in a small bowl mix the remaining cake mix oil water and eggs gently together in a medium bowl and whisk by hand or electric mixer for 23 minutes until smooth and creamy stir in the cake mix dusted chocolate chips place 75g of

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def sentence_similarity(sentence1, sentence2):
    # Tokenize the input sentences into words
    words1 = word_tokenize(sentence1)
    words2 = word_tokenize(sentence2)

    # Get a set of stopwords
    stop_words = set(stopwords.words('english'))

    # Remove stopwords from the word lists
    words1 = [word for word in words1 if word.lower() not in stop_words]
    words2 = [word for word in words2 if word.lower() not in stop_words]

    # Create a set of unique words from both sentences
    unique_words = set(words1 + words2)

    # Encode the sentences into binary vectors
    vector1 = [1 if word in words1 else 0 for word in unique_words]
    vector2 = [1 if word in words2 else 0 for word in unique_words]

    # Calculate the cosine similarity between the binary vectors
    dot_product = sum(x * y for x, y in zip(vector1, vector2))
    magnitude1 = sum(x ** 2 for x in vector1) ** 0.5
    magnitude2 = sum(x ** 2 for x in vector2) ** 0.5

    cosine_similarity = dot_product / (magnitude1 * magnitude2)

    return cosine_similarity

In [ ]:
import nltk
results1=[]
start_time_overall= time.time()
for i in range(len(Mainurl)):
    Url1text = Mainurl.loc[i, 'Url2text']
    Url1 = Mainurl.loc[i, 'Url2']
    for j in range(len(df)):
        Url2text = df.loc[j, 'Url2text']
        Url2 = df.loc[j, 'Url2']

        start_time = time.time()  # Start measuring time
        similarity1 = sentence_similarity(str(Url1text), str(Url2text))
        end_time = time.time()  # End measuring time
        execution_time1 = end_time - start_time

        result = {
            'Url1': Url1, 'Url1text': Url1text, 'Url2': Url2, 'Url2text': Url2text,
            'similarity': similarity1, 'time': execution_time1
        }
        results1.append(result)

end_time_overall = time.time()  # End measuring time
execution_time_overall = end_time_overall - start_time_overall

In [ ]:
execution_time_overall

448.784786939621

In [ ]:
import csv

with open('F733token1.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Url1","Url1text","Url2","Url2text", "Similarity","TimeComplexity"])

    for result in results1:
        try:
            writer.writerow([result["Url1"],result['Url1text'],result["Url2"], result['Url2text'], result['similarity'],result['time']])
        except UnicodeEncodeError:
            print(f"Skipping row with Url1text: {result['Url1text']} and Url2text: {result['Url2text']}")
